In [1]:
import os, re, json
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [2]:
# STEP 1 ─ Discover + Load + Merge ALL monthly CRMLS CSVs (no cleaning yet)

from pathlib import Path
import pandas as pd
import re

# ── Config ────────────────────────────────────────────────────────────────
DATA_DIR = Path("data")          # folder inside your "idx exchange" project
EXPLICIT_FILES = None            # optionally pin exact files (list of filenames) or leave as None
MONTH_RE = re.compile(r"CRMLSSold(\d{6})", re.IGNORECASE)  # extracts YYYYMM from names like CRMLSSold202508.csv

# ── Helper: find & sort monthly files by YYYYMM in the filename ──────────
def detect_monthly_files():
    if EXPLICIT_FILES:
        files = [DATA_DIR / f for f in EXPLICIT_FILES]
    else:
        files = [p for p in DATA_DIR.glob("CRMLSSold*.csv") if MONTH_RE.search(p.name)]
        # sort by the YYYYMM captured from filename
        files = sorted(files, key=lambda p: int(MONTH_RE.search(p.name).group(1)))
    if not files:
        raise FileNotFoundError("No CRMLSSold*.csv files found in ./data. Double-check the folder path and filenames.")
    return files

# ── Helper: robust loader (keeps everything raw; only parses date columns if present) ─
def load_one_month(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, low_memory=False)
    # standardize column whitespace
    df.columns = df.columns.str.strip()
    # Try to create/parse CloseDate using common variants present in different dumps
    if "CloseDate" in df.columns:
        df["CloseDate"] = pd.to_datetime(df["CloseDate"], errors="coerce")
    elif "CloseDateTime" in df.columns:
        df["CloseDate"] = pd.to_datetime(df["CloseDateTime"], errors="coerce")
    elif "CloseTimestamp" in df.columns:
        df["CloseDate"] = pd.to_datetime(df["CloseTimestamp"], errors="coerce")
    else:
        # If no date column exists (rare), derive a fallback from the filename so we can sort later
        m = MONTH_RE.search(path.name)
        if m:
            yyyymm = m.group(1)
            df["CloseDate"] = pd.to_datetime(f"{yyyymm[:4]}-{yyyymm[4:]}-01", errors="coerce")
        else:
            df["CloseDate"] = pd.NaT
    return df

# ── Discover files ────────────────────────────────────────────────────────
files = detect_monthly_files()
print("Found monthly files (sorted):")
for p in files:
    print(" •", p.name)

# ── Load & merge (no cleaning yet) ────────────────────────────────────────
parts = [load_one_month(p) for p in files]
df_all_raw = pd.concat(parts, ignore_index=True)
print(f"\nMerged raw shape: {df_all_raw.shape}")

# ── Quick sanity previews ─────────────────────────────────────────────────
print("\nColumns (first 25):", list(df_all_raw.columns[:25]))
print("\nFirst 3 rows:")
display(df_all_raw.head(3))

# month-wise counts (based on CloseDate) to confirm coverage
if "CloseDate" in df_all_raw.columns:
    counts_by_month = (
        df_all_raw.assign(_ym=df_all_raw["CloseDate"].dt.to_period("M"))
                  .groupby("_ym", dropna=False).size().sort_index()
    )
    print("\nRow counts by CloseDate month:")
    display(counts_by_month.tail(12))  # show recent 12 months
else:
    print("\n[warn] CloseDate not present/parsed; month summary skipped.")


Found monthly files (sorted):
 • CRMLSSold202502.csv
 • CRMLSSold202503.csv
 • CRMLSSold202504.csv
 • CRMLSSold202505.csv
 • CRMLSSold202506.csv
 • CRMLSSold202507.csv
 • CRMLSSold202508.csv

Merged raw shape: (156064, 78)

Columns (first 25): ['BuyerAgentAOR', 'ListAgentAOR', 'Flooring', 'ViewYN', 'WaterfrontYN', 'BasementYN', 'PoolPrivateYN', 'OriginalListPrice', 'ListingKey', 'ListAgentEmail', 'CloseDate', 'ClosePrice', 'ListAgentFirstName', 'ListAgentLastName', 'Latitude', 'Longitude', 'UnparsedAddress', 'PropertyType', 'LivingArea', 'ListPrice', 'DaysOnMarket', 'ListOfficeName', 'BuyerOfficeName', 'CoListOfficeName', 'ListAgentFullName']

First 3 rows:


,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,OriginalListPrice,ListingKey,ListAgentEmail,...,LotSizeDimensions,LotSizeArea,MainLevelBedrooms,NewConstructionYN,GarageSpaces,HighSchoolDistrict,PostalCode,AssociationFee,LotSizeSquareFeet,MiddleOrJuniorSchoolDistrict
0,RanchoSoutheast,RanchoSoutheast,NaN,True,NaN,NaN,NaN,60000.0,526199946,cmark1018@yahoo.com,...,NaN,28000.0,NaN,False,NaN,NaN,92307,0.0,28000.0,NaN
1,InlandValleys,InlandValleys,NaN,False,NaN,NaN,NaN,550000.0,525585060,mozcorona@aol.com,...,NaN,39640.0,NaN,False,NaN,NaN,92553,0.0,39640.0,NaN
2,SanDiego,SanDiego,NaN,False,NaN,NaN,False,880000.0,497696903,lenskab@gmail.com,...,NaN,NaN,NaN,False,2.0,NaN,91942,NaN,NaN,NaN



Row counts by CloseDate month:


_ym
2025-02    18702
2025-03    21445
2025-04    23262
2025-05    23154
2025-06    22883
2025-07    23646
2025-08    22972
Freq: M, dtype: int64

In [3]:
# Save merged raw dataset (optional, for audit/debug)
out_path = DATA_DIR / "all_raw.csv"
df_all_raw.to_csv(out_path, index=False)
print(f"\nMerged raw dataset saved to {out_path} with shape {df_all_raw.shape}")



Merged raw dataset saved to data/all_raw.csv with shape (156064, 78)


In [4]:
#Setup & helpers for cleaning

import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path("data")

# Columns to drop if present (leakage, marketing, process, non-predictive URLs/remarks)
DROP_COLUMNS = [
    "ListPrice","OriginalListPrice","ListingContractDate","DaysOnMarket","CumulativeDaysOnMarket",
    "Concessions","BuyerAgent","BuyerAgentName","BuyerOfficeName",
    "ListAgent","ListAgentName","ListAgentAOR","ListOfficeName","ListOfficeAOR",
    "CoListAgent","CoListOfficeName","ModificationTimestamp","PhotosCount",
    "PublicRemarks","PrivateRemarks","SyndicationRemarks","VirtualTourURL","ListingURL",
    "MlsStatus","OriginatingSystemName","OriginatingSystemID","ListingId","CloseComments"
]

# Core columns to keep if present (target/time + structure + location + IDs)
KEEP_CORE = [
    # target + time
    "ClosePrice","CloseDate",
    # structure
    "BedroomsTotal","BathroomsTotalInteger","BuildingAreaTotal",
    "LotSizeArea","LotSizeSquareFeet","YearBuilt","StoriesTotal",
    "GarageSpaces","CarportSpaces","CoveredSpaces","AttachedGarageYN",
    "ParkingFeatures","PoolFeatures","BasementYN",
    # location
    "PostalCode","City","CountyOrParish","Latitude","Longitude",
    "SubdivisionName","SchoolDistrict",
    # IDs/filters (kept for audit/traceability)
    "ListingKey","PropertyType","PropertySubType","StandardStatus"
]

# Numeric columns we will try to coerce + impute
NUMERIC_CANDIDATES = [
    "ClosePrice","BedroomsTotal","BathroomsTotalInteger","BuildingAreaTotal",
    "LotSizeArea","LotSizeSquareFeet","YearBuilt","StoriesTotal",
    "GarageSpaces","CarportSpaces","CoveredSpaces","Latitude","Longitude"
]

# Boolean-like columns to normalize to 0/1 if present
BOOL_LIKE = ["AttachedGarageYN","BasementYN"]

def scan(df: pd.DataFrame, name: str, n_cols: int = 20):
    """Quick schema scan: shape, nulls, dtypes (top N)."""
    print(f"\n=== {name}: shape {df.shape} ===")
    print("Null % (top):")
    print((df.isna().mean().sort_values(ascending=False).head(n_cols) * 100).round(1))
    print("\nDtypes (top):")
    print(df.dtypes.head(n_cols))


In [5]:
#Filter to Residential + SingleFamilyResidence

# Expect df_all_raw to be in memory from STEP 1
df_all_step = df_all_raw.copy()

# Standardize column whitespace just in case
df_all_step.columns = df_all_step.columns.str.strip()

# Keep Residential + SingleFamilyResidence (if cols exist; otherwise no-op)
if "PropertyType" in df_all_step.columns:
    df_all_step = df_all_step[
        df_all_step["PropertyType"].astype(str).str.strip().str.lower() == "residential"
    ]

if "PropertySubType" in df_all_step.columns:
    df_all_step = df_all_step[
        df_all_step["PropertySubType"].astype(str).str.strip().str.lower() == "singlefamilyresidence"
    ]

scan(df_all_step, "After SFR filter")



=== After SFR filter: shape (78387, 78) ===
Null % (top):
BusinessType                    100.0
CoveredSpaces                   100.0
MiddleOrJuniorSchoolDistrict    100.0
FireplacesTotal                 100.0
TaxAnnualAmount                 100.0
AboveGradeFinishedArea          100.0
TaxYear                         100.0
ElementarySchoolDistrict        100.0
WaterfrontYN                    100.0
BelowGradeFinishedArea           99.3
BasementYN                       97.6
BuilderName                      95.2
LotSizeDimensions                93.7
BuildingAreaTotal                93.3
CoBuyerAgentFirstName            90.9
ElementarySchool                 86.5
MiddleOrJuniorSchool             86.4
HighSchool                       82.4
CoListAgentFirstName             76.8
CoListAgentLastName              76.8
dtype: float64

Dtypes (top):
BuyerAgentAOR                 object
ListAgentAOR                  object
Flooring                      object
ViewYN                        object
Wat